In [4]:
'''This is the version that drops the rows'''

import dask.dataframe as dd

#Load the entire dataset with Dask
train_data = dd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet')

#Filter out the first 85 days
train_data = train_data[train_data['date_id'] >= 85]
#Drop fully empty (all-NaN) partitions to save memory
def drop_empty_partitions(df):
    return df.dropna(how='all')

train_data = train_data.map_partitions(drop_empty_partitions)

#Define a function to interpolate within each partition(moved outside the function for later data observation)
numerical_columns = train_data.select_dtypes(include=['number']).columns

# Updated interpolate function
def interpolate_partition(df):
    # Perform interpolation
    df[numerical_columns] = df[numerical_columns].interpolate(method='linear', limit_direction='both')
    return df

# Apply the interpolation function to each partition
train_data = train_data.map_partitions(interpolate_partition)
train_data = train_data.dropna(how='any')

#Verify if there are any remaining NaNs
remaining_na = train_data.isna().sum().sum().compute()
print("Remaining NaNs after interpolation:", remaining_na)


Remaining NaNs after interpolation: 0


In [6]:
# Compute summary statistics
summary = train_data[numerical_columns].describe(percentiles = [0.25, 0.5, 0.75]).compute()
print(summary)

            date_id       time_id     symbol_id        weight    feature_00  \
count  3.934201e+07  3.934201e+07  3.934201e+07  3.934201e+07  3.934201e+07   
mean   1.148453e+03  4.775525e+02  1.879362e+01  2.044000e+00  5.943514e-01   
std    3.312345e+02  2.767718e+02  1.122486e+01  1.143265e+00  1.346729e+00   
min    5.100000e+02  0.000000e+00  0.000000e+00  1.499667e-01 -5.794129e+00   
25%    8.710000e+02  2.580000e+02  1.100000e+01  1.476307e+00 -3.038115e-02   
50%    1.168000e+03  5.090000e+02  2.000000e+01  2.106626e+00  8.334433e-01   
75%    1.434000e+03  7.480000e+02  2.900000e+01  4.103159e+00  2.206827e+00   
max    1.698000e+03  9.670000e+02  3.800000e+01  1.024042e+01  6.477002e+00   

         feature_01    feature_02    feature_03    feature_04    feature_05  \
count  3.934201e+07  3.934201e+07  3.934201e+07  3.934201e+07  3.934201e+07   
mean   1.045648e-02  5.931975e-01  5.929216e-01  6.066390e-04 -4.811674e-02   
std    1.101107e+00  1.341614e+00  1.341667e+00  1.

In [7]:
# Compute IQR thresholds
q1 = train_data[numerical_columns].quantile(0.25).compute()
q3 = train_data[numerical_columns].quantile(0.75).compute()
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
print("Lower bounds:\n", lower_bound)
print("Upper bounds:\n", upper_bound)

Lower bounds:
 date_id         26.500000
time_id       -477.000000
symbol_id      -16.000000
weight          -0.930002
feature_00      -0.802240
                  ...    
responder_4     -1.362068
responder_5     -0.899112
responder_6     -1.467574
responder_7     -1.524877
responder_8     -1.376993
Length: 92, dtype: float64
Upper bounds:
 date_id        2278.500000
time_id        1483.000000
symbol_id        56.000000
weight            7.194063
feature_00        4.845968
                  ...     
responder_4       1.581917
responder_5       1.082564
responder_6       1.737236
responder_7       1.810217
responder_8       1.662059
Length: 92, dtype: float64


In [ ]:
import matplotlib.pyplot as plt

# Sample some data to visualize
sampled_data = train_data[numerical_columns].sample(frac=0.1).compute()

# Plot boxplots
sampled_data.boxplot(figsize=(12, 6))
plt.show()

In [ ]:
'''After applying the interpolation, I am still left with many NaN values. To tackle this,
I have two main courses of action, I can either completely drop these rows, or I can use median 
values as default placeholders. Although I believe that simply dropping these rows would be
the best strategy(given that they represent a very small portion of the data) There is no way 
to be sure.

because of this, i have decided to maintain two version of the data, one based on dropping nan 
rows, and the other based on median filling. I can cross validate these two later to deterine
                which is the best option to proceed with'''

In [ ]:
'''The code beyond this point is soley for the purpose of documenting my data exploration/experimentatin. 
although it is mostly the same logic as above, it cannot be guarenteed to run, nor can 
it be guarenteed to provide clean data.'''

In [ ]:
'''This is the version that fills remaining NaNs with medians'''

import dask.dataframe as dd

# Load the entire dataset with Dask
train_data = dd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet')

# Filter out the first 85 days
train_data = train_data[train_data['date_id'] >= 85]

# Drop fully empty (all-NaN) partitions to save memory
def drop_empty_partitions(df):
    return df.dropna(how='all')

train_data = train_data.map_partitions(drop_empty_partitions)

# Define a function to interpolate within each partition
def interpolate_partition(df):
    # Interpolate numerical columns only
    numerical_columns = df.select_dtypes(include=['number']).columns
    # Perform interpolation in multiple steps
    df[numerical_columns] = df[numerical_columns].interpolate(method='linear', limit_direction='both')
    return df

# Apply the interpolation function to each partition
train_data = train_data.map_partitions(interpolate_partition)

# Compute medians for numerical columns
# Sample a fraction of data to compute medians, adjusting `frac` as per memory constraints
sample_data = train_data.sample(frac=0.1).compute()  # Adjust `frac` as necessary
# Compute medians for numeric columns only
numeric_columns = sample_data.select_dtypes(include=['number'])
medians = numeric_columns.median()

# Fill remaining NaNs in the dataset with median values for numeric columns
train_data = train_data.map_partitions(
    lambda df: df.fillna(medians.to_dict())
)

# Optional: Verify if there are any remaining NaNs
remaining_na = train_data.isna().sum().sum().compute()
print("Remaining NaNs after interpolation and filling with medians:", remaining_na)


In [ ]:
''' this is no longer needed given that we are now using interprolation instead.

#Apply fillna with a limit to handle short gaps but retain larger missing sections
train_data = train_data.bfill(limit=5000).ffill(limit=5000)

#Sample 10% of the data to compute medians
sample_data = train_data.sample(frac=0.1).compute()

#Select only numeric columns
numeric_columns = sample_data.select_dtypes(include=['number'])
medians = numeric_columns.median()

# Step 4: Fill remaining NaNs using column-specific medians
train_data = train_data.fillna(medians.to_dict())

# Optional: Check for remaining NaNs if needed
remaining_na = train_data.isna().sum().sum().compute()
print("Remaining NaNs after fill:", remaining_na)



In [ ]:
import dask.dataframe as dd

# Load the dataset in Dask (memory-efficient)
data_path = '/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet'
train_data = dd.read_parquet(data_path)

# Take a 10% random sample of the data for initial inspection
sample_data = train_data.sample(frac=0.1, random_state=42).compute()  # Converts to pandas


In [ ]:

# Check the first few rows of the sample
print(sample_data.head())

# Check the data types of each column
#print(sample_data.dtypes)

# Get general information about the data, including non-null counts
#print(sample_data.info())


In [ ]:
# Get the total number of rows (compute once for efficiency)
total_rows = len(train_data)

# Count missing values in each column
missing_values = train_data.isna().sum().compute()  # .compute() to get concrete values in pandas

# Identify columns where the count of NaNs is equal to the total number of rows
all_nan_columns = missing_values[missing_values == total_rows].index
print("Columns with all NaN values:", all_nan_columns.tolist())

In [ ]:
#test what percent of each column is filled with NaN values
#will hopefully help determine which ones are causing the fill NaN error

import pandas as pd

# Calculate NaN percentages
nan_percentage = train_data.isna().mean().compute() * 100  # Compute to get concrete values in pandas

# Sort columns by NaN percentage in descending order (highest NaN percentage at the top)
nan_percentage_sorted = nan_percentage.sort_values(ascending=False)

# Display all columns with NaN percentages
pd.set_option('display.max_rows', None)  # Show all rows without truncation
print("NaN percentage per column (sorted):\n", nan_percentage_sorted)
pd.reset_option('display.max_rows')  # Reset display option back to default

'''
NaN percentage per column (sorted):
 feature_26      17.900406
feature_21      17.900406
feature_27      17.900406
feature_31      17.900406
feature_42       9.125593
feature_39       9.125593
feature_50       9.026816
feature_53       9.026816
feature_00       6.752030
feature_01       6.752030
feature_02       6.752030
feature_03       6.752030
feature_04       6.752030
feature_15       2.566024
feature_41       2.319274
feature_44       2.319274
feature_52       2.217180
feature_55       2.217180
feature_74       1.026493
feature_73       1.026493
'''


In [ ]:

# Count missing values in each column
missing_values = sample_data.isna().sum()
print("Missing values per column:\n", missing_values)

# Show only columns with missing values for better clarity
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values:\n", missing_columns)


In [ ]:
# Filter to exclude the first 85 days
sample_data = sample_data[sample_data['date_id'] >= 85]
print("Data after filtering the first 85 days:", sample_data.shape)


In [ ]:
# Apply backward-fill, then forward-fill
sample_data = sample_data.bfill().ffill()

# Verify that there are no remaining missing values
print("Remaining missing values:", sample_data.isna().sum().sum())  # Should be 0 if all NaNs are filled
